In [1]:
import segmentation_models as sm

Segmentation Models: using `keras` framework.


In [1]:
from keras_segmentation.models.unet import vgg_unet

In [2]:
n_classes = 23 # Aerial Semantic Segmentation Drone Dataset tree, gras, other vegetation, dirt, gravel, rocks, water, paved area, pool, person, dog, car, bicycle, roof, wall, fence, fence-pole, window, door, obstacle
model = vgg_unet(n_classes=n_classes ,  input_height=416, input_width=608  )

58892288/58889256 [==============================] - 13s 0us/step


In [3]:
model.train( 
    train_images =  "data/dataset/semantic_drone_dataset/original_images/",
    train_annotations = "data/dataset/semantic_drone_dataset/label_images_semantic/",
    checkpoints_path = "vgg_unet" , epochs=5
)

Verifying training dataset


100%|██████████| 400/400 [02:45<00:00,  2.42it/s]

Dataset verified! 
Instructions for updating:
Please use Model.fit, which supports generators.


Epoch 1/5
512/512 [==============================] - 421s 822ms/step - loss: 3.6572 - accuracy: 0.0346
Epoch 2/5
155/512 [========>.....................] - ETA: 4:57 - loss: 3.4281 - accuracy: 0.0686

KeyboardInterrupt: 

In [ ]:
tf.__version__

In [ ]:
df = pd.read_csv('~/Desktop/freelancer_drone/data/class_dict_seg.csv')
num_classes = df.shape[0]

In [ ]:
IMG_SIZE = 640

In [ ]:
input_dir = 'data/dataset/semantic_drone_dataset/original_images'
target_dir = 'data/dataset/semantic_drone_dataset/label_images_semantic'

In [ ]:
input_img_paths = sorted([i for i in Path(input_dir).glob('**/*.jpg')])
input_lbl_paths = sorted([i for i in Path(target_dir).glob('**/*.png')])

In [ ]:
i_ = np.zeros((10,) + (255, 255) + (1, ), dtype='uint8')
img = load_img('data/dataset/semantic_drone_dataset/label_images_semantic/491.png', target_size=(255,255), color_mode='grayscale')

In [ ]:
np.unique(np.expand_dims(img, 2))

In [ ]:
i_[0] = np.expand_dims(img, 2)

In [ ]:
np.unique(i_[0])

In [ ]:
class CDG(tf.keras.utils.Sequence):
    def __init__(self, batch_size, img_size, input_image_path, target_image_path, num_classes):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_image_path = input_image_path
        self.target_image_path = target_image_path
        self.num_classes = num_classes
    
    def __len__(self):
        return len(self.target_image_path) // self.batch_size
    
    def __random_flip(self, x, y):
        for i in range(x.shape[0]):
            print('flipped images')
            display(tf.keras.preprocessing.image.array_to_img(tf.image.flip_left_right((np.expand_dims(x[i],0))[0])))
            display(tf.keras.preprocessing.image.array_to_img(tf.image.flip_left_right((np.expand_dims(y[i],0))[0])))
        
    def __getitem__(self, idx):
        input_batch = self.input_image_path[idx*self.batch_size : idx*self.batch_size + self.batch_size]
        target_batch = self.target_image_path[idx*self.batch_size : idx*self.batch_size + self.batch_size]
        
        x = np.zeros((self.batch_size,) + self.img_size + (3, ), dtype='float32')
        for j, path in enumerate(input_batch):
            img = load_img(path, target_size=self.img_size)
            #img = np.asarray(img) / 255
            img = np.asarray(img)
            x[j] = img
            
        
        y = np.zeros((self.batch_size,) + self.img_size + (1, ), dtype='float32')
        for j, path in enumerate(target_batch):
            img = load_img(path, target_size=self.img_size, color_mode='grayscale')
            #img = np.asarray(img) / self.num_classes
            y[j] = np.expand_dims(img, 2)
            
        return x, y

In [ ]:
train_gen = CDG(3, (IMG_SIZE, IMG_SIZE), x_train, y_train, num_classes)
val_gen = CDG(3, (IMG_SIZE, IMG_SIZE), x_val, y_val, num_classes)

In [ ]:
np.unique(train_gen[0][1])

In [ ]:
model = sm.Unet('resnet34', encoder_weights='imagenet')
model.compile(
    'Adam',
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score],
)

# fit model
# if you use data generator use model.fit_generator(...) instead of model.fit(...)
# more about `fit_generator` here: https://keras.io/models/sequential/#fit_generator
model.fit(
   x=x_train,
   y=y_train,
   batch_size=16,
   epochs=100,
   validation_data=(x_val, y_val),
)

In [ ]:
preds = model.predict(val_gen)

In [ ]:
preds.shape

In [ ]:
def display_mask(i):
    """Quick utility to display a model's prediction."""
    mask = np.argmax(preds[i], axis=-1)
    print(np.unique(mask))
    mask = np.expand_dims(mask, axis=-1)
    img = PIL.ImageOps.autocontrast(tf.keras.preprocessing.image.array_to_img(mask))
    #img = tf.keras.preprocessing.image.array_to_img(mask)
    display(img)


# Display results for validation image #10
i = 5

# Display input image
display(Image(filename=x_val[i]))

# Display ground-truth target mask
img = PIL.ImageOps.autocontrast(load_img(y_val[i]))
display(img)

# Display mask predicted by our model
display_mask(i)  # Note that the model only sees inputs at 150x150.